<a href="https://colab.research.google.com/github/mlfa19/assignments/blob/master/Module%201/07/Assignment_07_Companion_Pytorch_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import gdown
import pandas as pd

gdown.download('https://drive.google.com/uc?authuser=0&id=1XIFiL3WxxR6M2nWgADi3xWvuRO6A-Ov8&export=download', 'titanic_train.csv', False)
df = pd.read_csv('titanic_train.csv')
df

Downloading...
From: https://drive.google.com/uc?authuser=0&id=1XIFiL3WxxR6M2nWgADi3xWvuRO6A-Ov8&export=download
To: /content/titanic_train.csv
100%|██████████| 61.2k/61.2k [00:00<00:00, 15.2MB/s]


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [0]:
# get rid of null values for age since this is just an illustrative example.
# this would not be a good thing to do if we were trying to evalutate the 
# performance of a model.
df_filtered = df[['Age', 'Sex', 'Survived']].dropna()
experiment_1_data = pd.concat((pd.get_dummies(df_filtered['Sex'], drop_first=True), df_filtered['Age']), axis=1)
experiment_1_outputs = df_filtered['Survived']

In [40]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', penalty='none') # setting solve silences annoying warning
model.fit(experiment_1_data, experiment_1_outputs)
print("coefs", model.coef_)
print("intercept", model.intercept_)

coefs [[-2.46592015 -0.0054259 ]]
intercept [1.27727321]


In [0]:
from torch import nn
import torch

class LogisticRegressionPT(nn.Module):
    def __init__(self):
        super(LogisticRegressionPT, self).__init__()
        self.linear = nn.Linear(2,1)

    def forward(self, X):
        X = self.linear(X)
        return torch.sigmoid(X)

In [14]:
import numpy as np
from torch.autograd import Variable

model_PT = LogisticRegressionPT()
model_PT.cuda()
optimizer = torch.optim.Adam(model_PT.parameters())
criterion = torch.nn.BCELoss()

X_data = Variable(torch.cuda.FloatTensor(np.array(experiment_1_data)))
y_data = Variable(torch.cuda.FloatTensor(np.array(experiment_1_outputs)))
for epoch in range(10000):
    model_PT.train()
    optimizer.zero_grad()
    # Forward pass
    y_pred = model_PT(X_data)
    # Compute Loss
    loss = criterion(y_pred, y_data)
    # Backward pass
    loss.backward()
    if epoch % 500 == 0:
        for name, param in model_PT.named_parameters():
            if param.requires_grad:
                print(name, "value", param.data, "gradient", param.grad)
    optimizer.step()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([714])) that is different to the input size (torch.Size([714, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


linear.weight value tensor([[ 0.2613, -0.2141]], device='cuda:0') gradient tensor([[ -0.1161, -11.3806]], device='cuda:0')
linear.bias value tensor([-0.6172], device='cuda:0') gradient tensor([-0.3827], device='cuda:0')
linear.weight value tensor([[ 0.0518, -0.0029]], device='cuda:0') gradient tensor([[ 0.1188, -0.1035]], device='cuda:0')
linear.bias value tensor([-0.4003], device='cuda:0') gradient tensor([-0.0174], device='cuda:0')
linear.weight value tensor([[-0.4224,  0.0038]], device='cuda:0') gradient tensor([[ 0.0896, -0.0340]], device='cuda:0')
linear.bias value tensor([-0.3283], device='cuda:0') gradient tensor([-0.0238], device='cuda:0')
linear.weight value tensor([[-0.8165,  0.0072]], device='cuda:0') gradient tensor([[ 0.0671, -0.0103]], device='cuda:0')
linear.bias value tensor([-0.2008], device='cuda:0') gradient tensor([-0.0262], device='cuda:0')
linear.weight value tensor([[-1.1570,  0.0082]], device='cuda:0') gradient tensor([[ 0.0495, -0.0002]], device='cuda:0')
linea

In [47]:
sample_point = X_data[0].cpu().detach().numpy()
sample_output = y_data[0].cpu().numpy()

optimizer.zero_grad()
y0 = model_PT(X_data[0])
loss = criterion(y0, y_data[0])
loss.backward()
print([p.grad for p in model_PT.linear.parameters()])

[tensor([[0.2128, 4.6820]], device='cuda:0'), tensor([0.2128], device='cuda:0')]


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:512: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


In [50]:
s = model.coef_.dot(sample_point)+model.intercept_
z = 1/(1+np.exp(-s))
print("gradient based on the maths!!", -(sample_output - z)*sample_point)

gradient based on the maths!! [0.21281891 4.68201593]
